<a href="https://colab.research.google.com/github/DaisyXinyiHe/diabetes_mellitus_predict/blob/pre_processing/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

## Execute this if using google colab and connecting it with google drive
## If not using google colab, navigate the script to your local drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Navigate data to its location
%cd /content/drive/MyDrive/Colab Notebooks/WiDS_2021

/content/drive/MyDrive/Colab Notebooks/WiDS_2021


In [3]:
# Load libraries
import numpy as np
import pandas as pd

In [109]:
# load training data

training_data = pd.read_csv('TrainingWiDS2021.csv')

In [110]:
# Explore dataset
print(training_data.head(1))
print('training data size is:', training_data.shape)

   Unnamed: 0  encounter_id  ...  solid_tumor_with_metastasis  diabetes_mellitus
0           1        214826  ...                            0                  1

[1 rows x 181 columns]
training data size is: (130157, 181)


In [111]:
# Check if feature variables have NA values
training_data = training_data.iloc[:,1:]
training_data.isnull().any()

encounter_id                   False
hospital_id                    False
age                             True
bmi                             True
elective_surgery               False
                               ...  
immunosuppression              False
leukemia                       False
lymphoma                       False
solid_tumor_with_metastasis    False
diabetes_mellitus              False
Length: 180, dtype: bool

In [112]:
# check data types
print(training_data.dtypes)

encounter_id                     int64
hospital_id                      int64
age                            float64
bmi                            float64
elective_surgery                 int64
                                ...   
immunosuppression                int64
leukemia                         int64
lymphoma                         int64
solid_tumor_with_metastasis      int64
diabetes_mellitus                int64
Length: 180, dtype: object


In [113]:
# check how many unique data types is in training_data
np.unique(training_data.dtypes)

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

In [114]:
# Check categorical data
for c in training_data.columns:
  if training_data[c].dtypes == np.dtype('O'):
      print(c)

ethnicity
gender
hospital_admit_source
icu_admit_source
icu_stay_type
icu_type


In [115]:
# Check if independent variable have nan
print(training_data['diabetes_mellitus'].isnull().any)

<bound method Series.any of 0         False
1         False
2         False
3         False
4         False
          ...  
130152    False
130153    False
130154    False
130155    False
130156    False
Name: diabetes_mellitus, Length: 130157, dtype: bool>


In [116]:
print(training_data.columns)

Index(['encounter_id', 'hospital_id', 'age', 'bmi', 'elective_surgery',
       'ethnicity', 'gender', 'height', 'hospital_admit_source',
       'icu_admit_source',
       ...
       'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min', 'aids', 'cirrhosis',
       'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'diabetes_mellitus'],
      dtype='object', length=180)


In [117]:
# Assumming we want to keep all variables as predictors, 
# we need to create dummy variable all variables with NA values,  
# and change NA values to 0
# # Only do it to non-string variable for now
# Exclude ids and diabetes mellitus
for c in training_data.columns[3:]:
  if training_data[c].isnull().any() and training_data[c].dtypes != np.dtype('O'):


    newcol = c+'_0_1'
    newcol_value = np.array(training_data[c])


    nanindex = training_data[c].index[training_data[c].apply(np.isnan)]

    newcol_value[nanindex] = 0

    newcol_value[newcol_value!= 0]  = 1

    training_data[str(newcol)] = newcol_value


In [118]:
print(training_data.columns)

Index(['encounter_id', 'hospital_id', 'age', 'bmi', 'elective_surgery',
       'ethnicity', 'gender', 'height', 'hospital_admit_source',
       'icu_admit_source',
       ...
       'd1_pao2fio2ratio_max_0_1', 'd1_pao2fio2ratio_min_0_1',
       'h1_arterial_pco2_max_0_1', 'h1_arterial_pco2_min_0_1',
       'h1_arterial_ph_max_0_1', 'h1_arterial_ph_min_0_1',
       'h1_arterial_po2_max_0_1', 'h1_arterial_po2_min_0_1',
       'h1_pao2fio2ratio_max_0_1', 'h1_pao2fio2ratio_min_0_1'],
      dtype='object', length=335)


In [119]:
# # Check categorical data (e.g. ethnicity or dtype('O'))
for c in training_data.columns:
  if training_data[c].isnull().any() and training_data[c].dtypes == np.dtype('O'):
    print(c)
    print(training_data[c].unique())
    print(len(training_data[c].index[training_data[c].apply(pd.isnull)]))

ethnicity
['Caucasian' nan 'Hispanic' 'African American' 'Asian' 'Native American'
 'Other/Unknown']
1587
gender
['M' 'F' nan]
66
hospital_admit_source
['Floor' 'Emergency Department' 'Operating Room' nan 'Direct Admit'
 'Other Hospital' 'Other ICU' 'ICU to SDU' 'Recovery Room'
 'Chest Pain Center' 'Step-Down Unit (SDU)' 'Acute Care/Floor' 'PACU'
 'Observation' 'ICU' 'Other']
33198
icu_admit_source
['Floor' 'Accident & Emergency' 'Operating Room / Recovery'
 'Other Hospital' 'Other ICU' nan]
240


In [120]:
# Taking out "hospital_admit_source" and "icu_admit_source" because they don't seem to be relevant 
# to the prediction of diabete mellitus, and they have a lot of missing data
training_data = training_data.drop(columns=['icu_admit_source', 'hospital_admit_source'])


In [121]:
# Putting all "nan" in ethnicity to "other/unknown"
nanindex = training_data['ethnicity'].index[training_data['ethnicity'].apply(pd.isnull)]
training_data['ethnicity'].iloc[nanindex] = 'Other/Unknown'


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [122]:
# Delete the rows if they have nan values in "gender". This is only 66 rows
nanindex = training_data['gender'].index[training_data['gender'].apply(pd.isnull)]
training_data = training_data.drop(nanindex)
print(training_data.shape)


(130091, 333)


In [123]:
# Check all current categorical data
for c in training_data.columns:
  if training_data[c].dtypes == np.dtype('O'):
      print(c)



ethnicity
gender
icu_stay_type
icu_type


In [125]:
# Get dummy variable for categorical data
# dummy_v = pd.get_dummies(training_data[['ethnicity','gender','icu_stay_type','icu_type']],drop_first=True)
training_data = pd.get_dummies(data=training_data, columns=['ethnicity','gender','icu_stay_type','icu_type'])

In [126]:
training_data.head(5)

,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,...,h1_potassium_min_0_1,h1_sodium_max_0_1,h1_sodium_min_0_1,h1_wbc_max_0_1,h1_wbc_min_0_1,d1_arterial_pco2_max_0_1,d1_arterial_pco2_min_0_1,d1_arterial_ph_max_0_1,d1_arterial_ph_min_0_1,d1_arterial_po2_max_0_1,d1_arterial_po2_min_0_1,d1_pao2fio2ratio_max_0_1,d1_pao2fio2ratio_min_0_1,h1_arterial_pco2_max_0_1,h1_arterial_pco2_min_0_1,h1_arterial_ph_max_0_1,h1_arterial_ph_min_0_1,h1_arterial_po2_max_0_1,h1_arterial_po2_min_0_1,h1_pao2fio2ratio_max_0_1,h1_pao2fio2ratio_min_0_1,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_F,gender_M,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU
0,214826,118,68.0,22.732803,0,180.3,92,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0
1,246060,81,77.0,27.421875,0,160.0,90,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
2,276985,118,25.0,31.952749,0,172.7,93,0.000694,0,95.3,NaN,122.0,703.03,0,0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0,68.0,NaN,NaN,NaN,NaN,37.0,NaN,36.7,NaN,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
3,262220,118,81.0,22.635548,1,165.1,92,0.000694,0,61.7,NaN,203.0,1206.03,1,0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1,60.0,30.0,30.0,142.0,7.39,4.0,NaN,34.8,NaN,1,8.0,62.0,30.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0
4,201746,33,19.0,NaN,0,188.0,91,0.073611,0,NaN,NaN,119.0,601.01,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0,103.0,NaN,NaN,NaN,NaN,16.0,NaN,36.7,NaN,0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0


In [127]:
# Save cleaned training dataset
training_data.to_csv('training_data_clean.csv',index=False)

In [128]:
# Define independent and dependent variables
y = training_data['diabetes_mellitus']
training_data = training_data.drop(columns=['diabetes_mellitus'])
# encounter_id and hospital_id should not be invluded as predictors
training_data = training_data.drop(columns=['encounter_id', 'hospital_id'])
print(y.shape)
print(training_data.shape)

(130091,)
(130091, 345)
